# Simplified AES Key Recovery: 1st bit

## Description

In this attack, we aim to recover the first bit of the key using a binary classifier. We train a neural network on plaintext-ciphertext pairs as samples and the first bit of the key as labels. We use the educational simplified AES, which has only 16 bits keys and 16 bits blocks.

## Imports

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from dataset.datasets import SimplifiedAESDatasetCiphertextPlaintextPairKey
from pipeline import *

2023-05-02 16:21:42.203469: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Importing the dataset

In [2]:
data = SimplifiedAESDatasetCiphertextPlaintextPairKey('large')

train_labels, train_samples, test_labels, test_samples = data.get_data()

Error: dataset/assets/moby-dick.txt
Error: dataset/assets/shakespeare.txt


In [3]:
get_dataset_info(train_labels, train_samples, test_labels, test_samples)

===== Training Labels Shape: (11565605, 16)
===== Label Shape: (16,)
===== Training Samples Shape: (11565605, 32)
===== Sample Shape: (32,)
===== Testing Labels Shape: (4956688, 16)
===== Testing Samples Shape: (4956688, 32)


In [4]:
train_samples = train_samples.astype(float)
test_samples = test_samples.astype(float)

### Preprocessing

We add 1 preprocessing step:
As our goal is only to recover the first bit of the key, we only keep the first bit of the labels to train our network.


In [5]:
train_labels_1st_bit = np.array([l[0] for l in train_labels]).astype(float)
test_labels_1st_bit = np.array([l[0] for l in test_labels]).astype(float)

In [6]:
get_dataset_info(train_labels_1st_bit, train_samples, test_labels_1st_bit, test_samples)

===== Training Labels Shape: (11565605,)
===== Label Shape: ()
===== Training Samples Shape: (11565605, 32)
===== Sample Shape: (32,)
===== Testing Labels Shape: (4956688,)
===== Testing Samples Shape: (4956688, 32)


## Creating the model

In [7]:
# Imports
from keras import Sequential
from keras.layers import Input, Dense, BatchNormalization, LayerNormalization
from keras.optimizers import Adam

### Model hyperparameters
In this code block, we specify most parameters and hyperparameters that will be used in the training of the neural network.

Add customization here.

In [8]:
input_shape = np.shape(train_samples[0])

# output dimension
dim = 1

# units per hidden layer
units = 256

# lr_schedule = keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate=0.001,
#     decay_steps=10000,
#     decay_rate=-0.9)

loss_scc = 'sparse_categorical_crossentropy'
loss_mse = 'mse'
loss_bce = 'binary_crossentropy'
learning_rate = 0.001
optimizer = Adam(learning_rate=learning_rate)
metrics = ['binary_accuracy']
epochs = 50
batch_size = 5000

### Model
In this code block, we create the model, according to the parameters and the topology we want to achieve. 
We then compile it specifying the optimizer, the loss and the metrics we want outputted.

Add customization here.

In [9]:
# Type of model
neural_network = Sequential()

# Input layer
neural_network.add(Input(shape=input_shape))

# Hidden layers
#neural_network.add(BatchNormalization())
neural_network.add(Dense(units=units, activation='relu'))
neural_network.add(Dense(units=units, activation='relu'))
neural_network.add(Dense(units=units, activation='relu'))

# Output layer
neural_network.add(Dense(units=dim, activation='sigmoid'))

# Summary
neural_network.summary()

# Compile model
neural_network.compile(optimizer=optimizer, loss=loss_bce, metrics=metrics)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               8448      
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dense_2 (Dense)             (None, 256)               65792     
                                                                 
 dense_3 (Dense)             (None, 1)                 257       
                                                                 
Total params: 140,289
Trainable params: 140,289
Non-trainable params: 0
_________________________________________________________________


### Training
In this code block, we train the model. It outputs, for each epoch, the loss and metrics.

This block mostly stays the same.

In [10]:
history = train_model(neural_network, train_samples, train_labels_1st_bit, 
                      batch_size=batch_size, 
                      epochs=epochs)

Epoch 1/50
2314/2314 [==============================] - 63s 26ms/step - loss: 0.6932 - binary_accuracy: 0.5018
Epoch 2/50
2314/2314 [==============================] - 60s 26ms/step - loss: 0.6931 - binary_accuracy: 0.5036
Epoch 3/50
2314/2314 [==============================] - 59s 26ms/step - loss: 0.6930 - binary_accuracy: 0.5062
Epoch 4/50
2314/2314 [==============================] - 56s 24ms/step - loss: 0.6927 - binary_accuracy: 0.5094
Epoch 5/50
2314/2314 [==============================] - 56s 24ms/step - loss: 0.6921 - binary_accuracy: 0.5139
Epoch 6/50
2314/2314 [==============================] - 57s 25ms/step - loss: 0.6910 - binary_accuracy: 0.5193
Epoch 7/50
2314/2314 [==============================] - 58s 25ms/step - loss: 0.6894 - binary_accuracy: 0.5252
Epoch 8/50
2314/2314 [==============================] - 57s 25ms/step - loss: 0.6873 - binary_accuracy: 0.5309
Epoch 9/50
2314/2314 [==============================] - 58s 25ms/step - loss: 0.6850 - binary_accuracy: 0.5362
E

### Testing
Here, we evaluate the neural network with the test data.

This block stays the same.

In [11]:
results = test_model_binary(neural_network, test_samples, test_labels_1st_bit, batch_size)

992/992 [==============================] - 8s 8ms/step


In [48]:
print("Correct predictions: " + str(results["correct_predictions"]))
print("Accuracy: " + str(results["accuracy"]))

Correct predictions: 333625
Accuracy: 0.5133110853654221


In [18]:
total_1_bits = sum(test_labels_1st_bit)
distr = total_1_bits / len(test_labels_1st_bit)
print("'1' first bit distribution: " + str(distr))
print("'0' first bit distribution: "+ str(1-distr))

'1' first bit distribution: 0.5022962913945764
'0' first bit distribution: 0.49770370860542357


### Saving the model

In [17]:
save_model(neural_network, "s_aes_1st_bit")